In [1]:
%run ../__init__edwh__new.ipynb

Python 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
Setting up for long running connection


/shared_code/edwh/core/pgcache/__init__.py:91: UserWarning: Magic aangeroepn, dat klopt nie. 
  warnings.warn("Magic aangeroepn, dat klopt nie. ")


 * postgresql://postgres@pgpool:5432/backend
Done.


In [2]:
db(db.item).select(db.item.short_description, limitby=(10, 11)).first()

<Row {'short_description': 'bla'}>

In [3]:
print(db(db.item.gid == '2688c487-3d75-4356-b976-c7023de7c012').select(db.item.short_description).first().short_description)

Op [Liemers College](https://www.liemerscollege.nl/Home) in Zevenaar wordt vanaf 2014 samengewerkt met vier andere scholen binnen het leerlab 'Verbinding curriculum, content en platform.' 


**Aanleiding**


Het doel van de samenwerking 'Verbinding curriculum, content en platform' is het verkennen van manieren om leermateriaal, het curriculum en digitale platforms met elkaar te verbinden, met als uiteindelijk doel het leerproces van studenten effectiever en transparanter te maken. Zo kunnen de scholen beter het leerproces van de studenten monitoren, flexibeler inspelen op de individuele behoeften en een meer gepersonaliseerd leertraject aanbieden.



**Werkwijze** 


Om deze doelstelling te bereiken, heeft het Liemers College samen met de andere scholen binnen het leerlab verschillende vormen van online leermateriaal ontwikkeld en uitgewisseld. Hierdoor hadden ze toegang tot een breder scala aan onderwijsmateriaal, wat hen in staat stelde om hun eigen onderwijs flexibeler vorm te geven

In [4]:
(
    db(db.item.short_description.ilike("%**Resultaten**%")).count(),
    db(db.item.short_description.ilike("%Resultaten%")).count()
)

(1213, 1231)

In [5]:
query = db.item.short_description.ilike("%Resultaten%")
query &= ~db.item.short_description.ilike("%**Resultaten**%")
query &= ~db.item.short_description.ilike("%**Resultaten:**%")
query &= ~db.item.short_description.ilike("%**Resultaten **%")
query &= ~db.item.short_description.ilike("%**Resulaten**%")
query &= ~db.item.short_description.ilike("%**Resultaat**%")

rows = db(query).select(db.item.gid, db.item.short_description)
len(rows)

6

In [6]:
for row in rows:
    print(row.gid, row.short_description)
    print("------------------")

1ed41a21-2b96-43ca-9187-16fb830a07b8 Op [KC Wilp-Achterhoek](https://www.kcwilpachterhoek.nl/) in Wilp-Achterhoek wordt vanaf 2018 gewerkt met een rapportfolio. 

**Aanleiding**

Bij ons staat ontwikkelingsgericht leren centraal: ‘Ik ontdek, ik groei en ik ben’. Het gaat om de ontwikkeling van de leerling op basis van o.a. leerdoelen. Hoe dat verloopt, houden we niet bij in een klassiek rapport met cijfers, maar in een vorm die hier veel beter op aansluit: een rapportfolio.

**Werkwijze**

Dit rapportfolio is een map waarin de leerlingen werkjes, materialen en foto’s verzamelen waar ze trots op zijn. Omdat we de leerlingen een doorlopende leerlijn bieden van 2 tot 13 jaar, krijgen leerlingen vanaf de peutergroep een rapportfolio. Het rapportfolio wordt een verslag waarin beschreven staat wat de leerling ontdekt, doet en leert en laat daarmee de ontwikkeling zien. 

Hoe ziet die ontwikkeling eruit? Met andere woorden: wat zijn de leerdoelen die we voor deze leerling voor ogen hebben? Di

In [7]:
# omzetten

In [12]:
from tqdm import tqdm

sources = [
    "**Resultaten**",
    "**Resultaten:**",
    "**Resultaten **",
    "**Resulaten**",
    "**Resultaat**"
]

target = "**Opbrengsten**"

query = db.item.id == 0

for src in sources:
    query |= db.item.short_description.ilike(f"%{src}%")

count = db(query).count()
rows = db(query).iterselect(db.item.id, db.item.short_description)

for row in tqdm(rows, total=count):
    description = row.short_description
    for src in sources:
        description = description.replace(src, target)

    if description == row.short_description:
        raise ValueError("Nothing changed!")

    row.update_record(short_description=description)
else:
    print("Nothing to do!")

db.commit()

0it [00:00, ?it/s]

Nothing to do!


In [14]:
from edwh.core.pgcache.clean import clean

clean()

Connecting to database ...
Connected, truncating ...
done. 
